In [1]:
from Process import PlateReader
import pandas as pd

# Step 1: Load Sample Information
* Ideally will be in the form of a dataframe. This dataframe should be standardized in the original documentation where we originally made the sample. 
* Either load locally or pull from the google team drive (still working on fixing pulling from Drive)

In [26]:
sample_info = pd.read_csv(r"C:\Users\Edwin\Desktop\OT2-DOE\PlanPrepareProcess\03_08_21 Experiment Synthesis Info")
sample_info

,UID,dppc wtf,dspepeg200 wtf,pfh wtf,ethanol wtf,water wtf,Well,Labware,Slot,dppc-ethanol-stock uL,dspepeg2000-ethanol-stock uL,pfh-ethanol-stock uL,ethanol-stock uL,water-stock uL
0,S2_A1_03-10-2021_Left,0.0001,0.000035,0.0,0.222222,0.777643,A1,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,62.377473,0.0,184.469269,1166.464167
1,S2_A2_03-10-2021_Left,0.0001,0.000035,0.0,0.333333,0.666532,A2,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,62.377473,0.0,336.064013,999.797500
2,S2_A3_03-10-2021_Left,0.0001,0.000035,0.0,0.444444,0.555421,A3,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,62.377473,0.0,479.740702,833.130833
3,S2_A4_03-10-2021_Left,0.0001,0.000035,0.0,0.555556,0.444309,A4,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,62.377473,0.0,614.311941,666.464167
4,S2_A5_03-10-2021_Left,0.0001,0.000035,0.0,0.666667,0.333198,A5,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,62.377473,0.0,738.832881,499.797500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,S5_C8_03-10-2021_Left,0.0000,0.000000,0.0,0.445579,0.554421,C8,Falcon 48 Well Plate 1500 ÂµL,5,0.000000,0.000000,0.0,618.540719,831.630833
120,S5_D1_03-10-2021_Left,0.0000,0.000000,0.0,0.556691,0.443309,D1,Falcon 48 Well Plate 1500 ÂµL,5,0.000000,0.000000,0.0,753.013533,664.964167
121,S5_D2_03-10-2021_Left,0.0000,0.000000,0.0,0.667802,0.332198,D2,Falcon 48 Well Plate 1500 ÂµL,5,0.000000,0.000000,0.0,877.428149,498.297500
122,S5_D3_03-10-2021_Left,0.0000,0.000000,0.0,0.778913,0.221087,D3,Falcon 48 Well Plate 1500 ÂµL,5,0.000000,0.000000,0.0,991.231205,331.630833


# Step 2: Load UV-Vis Data
* ***If loading from plate reader***: Once the plate has been read, allow for exporting to an excel sheet and reformat for easy upload and merging.
    * The plate reader should automatically ask you if you would like to export to excel. When running multiple plates you can just keep selecting export and it will automatically add the new run as an additional sheet to the originally generated excel sheet. 
    * To format for easy loading:
        1. Create a new sheet in the same exported excel file and name it something simple (i.e. sheet1). 
        2. Copy the data from the sheet and add it to the newly created sheet. Make sure to paste in the data as "only values". 
        3. Repeat for every single exported sheet (i.e. every plate)
        
        
* Sometimes when using the plate reader the scanning protocol scans the whole plate versus only the wells of interest. This can be changed, however it is common to forget. If you do forget to do this, make sure to trim the dataframe to match the same length/order of wells as the sample info loaded in Step 1.


* ***If loading from single cuvette reader***: 
    * Still documenting.

***Step 2a:*** Provide the path of the excel sheet for the plate reader. With this you will also need to provide the **Sheet Name** of for correct excel sheet to be extracted. Since multiple plates are expected to be used at once, provide the Sheet names as a ***list of string names***. This will return a list of dataframes of each plate in the order the sheet names provided.

*Hmm maybe make dataframes as dictionary so you can look and debug later*

In [27]:
path = r"C:\Users\Edwin\Downloads\03_08_21_Left_Robot_P12.xlsx"
plate_names = ['Sheet1','Sheet2']
plate_dfs = PlateReader.extract_plates(path, plate_names)

***Step 2b:*** With the dataframes from ***Step2a***, given that all the plates read originally were done using the same protcol, then we can merge these dataframes into one. The only thing we need to edit is when originally formatting the excel sheets, since we copied the wavelength row in each sheet it only needs to be present once. A way to avoid to this is to copy it only in the first plate sheet you are using. 

In [28]:
merged_df = PlateReader.merge_wavelength_dfs(plate_dfs)

# Trim if needed
merged_df = merged_df[:len(sample_info)+1]
merged_df

,0,1,2,3,4,5,6,7,8,9,10,11,12
Wavelength,300.000,350.000,400.000,450.000,500.000,550.000,600.000,650.000,700.000,750.000,800.000,850.000,900.000
A1,0.304,0.075,0.051,0.048,0.046,0.046,0.047,0.046,0.046,0.047,0.047,0.048,0.048
A2,0.340,0.105,0.079,0.071,0.066,0.066,0.066,0.068,0.068,0.075,0.072,0.075,0.095
A3,0.308,0.080,0.053,0.048,0.045,0.043,0.042,0.040,0.039,0.044,0.043,0.047,0.063
A4,0.317,0.079,0.052,0.047,0.044,0.044,0.044,0.043,0.042,0.046,0.045,0.048,0.065
...,...,...,...,...,...,...,...,...,...,...,...,...,...
B12,0.298,0.066,0.042,0.041,0.038,0.038,0.038,0.037,0.037,0.043,0.043,0.047,0.067
C1,0.288,0.064,0.042,0.041,0.041,0.041,0.041,0.040,0.040,0.046,0.045,0.048,0.069
C2,0.279,0.064,0.041,0.040,0.038,0.038,0.039,0.038,0.038,0.043,0.042,0.044,0.066
C3,0.290,0.065,0.042,0.040,0.039,0.038,0.038,0.038,0.038,0.042,0.041,0.043,0.067


***Step 2c:*** Sometimes when samples are extremely turbid or have some sort of material like foam or cream the plate reader absorbance is overflowed and returns as 'OVRFLW'. This will prevent you from handling the entire set of data, so what can be done is replace the 'OVRFLW' values with a holder number. It is reccomended you select a very high holder number so you can track and isolate it easier if needed. 

In [29]:
merged_df = PlateReader.detect_ovflw(merged_df, holder = 30) # should have something to detect OVERFLW and call it out
# merged_df.drop(['Wavelength'], inplace = True)
merged_df

,0,1,2,3,4,5,6,7,8,9,10,11,12
Wavelength,300.000,350.000,400.000,450.000,500.000,550.000,600.000,650.000,700.000,750.000,800.000,850.000,900.000
A1,0.304,0.075,0.051,0.048,0.046,0.046,0.047,0.046,0.046,0.047,0.047,0.048,0.048
A2,0.340,0.105,0.079,0.071,0.066,0.066,0.066,0.068,0.068,0.075,0.072,0.075,0.095
A3,0.308,0.080,0.053,0.048,0.045,0.043,0.042,0.040,0.039,0.044,0.043,0.047,0.063
A4,0.317,0.079,0.052,0.047,0.044,0.044,0.044,0.043,0.042,0.046,0.045,0.048,0.065
...,...,...,...,...,...,...,...,...,...,...,...,...,...
B12,0.298,0.066,0.042,0.041,0.038,0.038,0.038,0.037,0.037,0.043,0.043,0.047,0.067
C1,0.288,0.064,0.042,0.041,0.041,0.041,0.041,0.040,0.040,0.046,0.045,0.048,0.069
C2,0.279,0.064,0.041,0.040,0.038,0.038,0.039,0.038,0.038,0.043,0.042,0.044,0.066
C3,0.290,0.065,0.042,0.040,0.039,0.038,0.038,0.038,0.038,0.042,0.041,0.043,0.067


# Step 3: Merge with Sample Info Dataframe

The first step to merge to sample info dataframe is to format the plate reader dataframe to have the wavelengths as the column names versus as an another row. We can do this by isolating the Wavelength row, adding the appropiate unit the string of each element of the wavlength row and reassigning this new list as the column names. We finalize this by dropping the wavlength row.

In [30]:
platereader_df = PlateReader.merge_sampleinfo_platereader(sample_info, merged_df)
platereader_df

,300.0nm,350.0nm,400.0nm,450.0nm,500.0nm,550.0nm,600.0nm,650.0nm,700.0nm,750.0nm,800.0nm,850.0nm,900.0nm
A1,0.304,0.075,0.051,0.048,0.046,0.046,0.047,0.046,0.046,0.047,0.047,0.048,0.048
A2,0.340,0.105,0.079,0.071,0.066,0.066,0.066,0.068,0.068,0.075,0.072,0.075,0.095
A3,0.308,0.080,0.053,0.048,0.045,0.043,0.042,0.040,0.039,0.044,0.043,0.047,0.063
A4,0.317,0.079,0.052,0.047,0.044,0.044,0.044,0.043,0.042,0.046,0.045,0.048,0.065
A5,0.330,0.075,0.049,0.046,0.044,0.043,0.042,0.041,0.040,0.044,0.044,0.047,0.066
...,...,...,...,...,...,...,...,...,...,...,...,...,...
B12,0.298,0.066,0.042,0.041,0.038,0.038,0.038,0.037,0.037,0.043,0.043,0.047,0.067
C1,0.288,0.064,0.042,0.041,0.041,0.041,0.041,0.040,0.040,0.046,0.045,0.048,0.069
C2,0.279,0.064,0.041,0.040,0.038,0.038,0.039,0.038,0.038,0.043,0.042,0.044,0.066
C3,0.290,0.065,0.042,0.040,0.039,0.038,0.038,0.038,0.038,0.042,0.041,0.043,0.067


Next we reset both indexes of the two dataframes we are to merge and concatnate them together. We drop and reset indexes to ensure any past dependency is forgotten, so you must make sure your sample info dataframe and plate reader dataframe are ordered correctly where row 1...n of one dataframe referes to row 1...n of the other. 

In [31]:
sample_info.reset_index(drop=True, inplace=True)
platereader_df.reset_index(drop=True, inplace=True)
combined_df = pd.concat([sample_info, platereader_df], axis = 1)
combined_df

,UID,dppc wtf,dspepeg200 wtf,pfh wtf,ethanol wtf,water wtf,Well,Labware,Slot,dppc-ethanol-stock uL,...,450.0nm,500.0nm,550.0nm,600.0nm,650.0nm,700.0nm,750.0nm,800.0nm,850.0nm,900.0nm
0,S2_A1_03-10-2021_Left,0.0001,0.000035,0.0,0.222222,0.777643,A1,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,...,0.048,0.046,0.046,0.047,0.046,0.046,0.047,0.047,0.048,0.048
1,S2_A2_03-10-2021_Left,0.0001,0.000035,0.0,0.333333,0.666532,A2,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,...,0.071,0.066,0.066,0.066,0.068,0.068,0.075,0.072,0.075,0.095
2,S2_A3_03-10-2021_Left,0.0001,0.000035,0.0,0.444444,0.555421,A3,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,...,0.048,0.045,0.043,0.042,0.040,0.039,0.044,0.043,0.047,0.063
3,S2_A4_03-10-2021_Left,0.0001,0.000035,0.0,0.555556,0.444309,A4,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,...,0.047,0.044,0.044,0.044,0.043,0.042,0.046,0.045,0.048,0.065
4,S2_A5_03-10-2021_Left,0.0001,0.000035,0.0,0.666667,0.333198,A5,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,...,0.046,0.044,0.043,0.042,0.041,0.040,0.044,0.044,0.047,0.066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,S5_C8_03-10-2021_Left,0.0000,0.000000,0.0,0.445579,0.554421,C8,Falcon 48 Well Plate 1500 ÂµL,5,0.000000,...,0.041,0.038,0.038,0.038,0.037,0.037,0.043,0.043,0.047,0.067
120,S5_D1_03-10-2021_Left,0.0000,0.000000,0.0,0.556691,0.443309,D1,Falcon 48 Well Plate 1500 ÂµL,5,0.000000,...,0.041,0.041,0.041,0.041,0.040,0.040,0.046,0.045,0.048,0.069
121,S5_D2_03-10-2021_Left,0.0000,0.000000,0.0,0.667802,0.332198,D2,Falcon 48 Well Plate 1500 ÂµL,5,0.000000,...,0.040,0.038,0.038,0.039,0.038,0.038,0.043,0.042,0.044,0.066
122,S5_D3_03-10-2021_Left,0.0000,0.000000,0.0,0.778913,0.221087,D3,Falcon 48 Well Plate 1500 ÂµL,5,0.000000,...,0.040,0.039,0.038,0.038,0.038,0.038,0.042,0.041,0.043,0.067


# Step 4: Format Dataframe as csv and Upload to Drive.
We simply call a method of the dataframe while ensuring to drop the index (again to remove and previous dependency). 

***If uploading from desktop***: Just grab and upload like any other file.

***If uploading from jupyter notebook***: Save the csv anywhere with a name that is not relevant, reccomended is something like "Temp File" as you will be uploading it to google drive under a different name. Then use the Gdrive module of functions from the Prepare package. Read the README in this package to understand how to use it, it will require a certain setup of credentials. 


In [32]:
temp_hold_path = r"03_08_21_Left_UVVis"
combined_df_csv = combined_df.to_csv(temp_hold_path, index=False)

In [9]:
from Prepare import CreateSamples

InvalidConfigError: Invalid client secrets file ('Error opening file', 'client_secrets.json', 'No such file or directory', 2)

In [ ]:
CreateSamples.team_drive_dict()

In [ ]:
current_members = CreateSamples.file_and_folder_navi('1dXYmxuESNhgmVHntEf8hnZmjK0Br8b64')
CreateSamples.file_and_folder_navi('1CDnoLE32bG0BFhhvCl5JlRFO4I_ga-eJ')

In [ ]:
file_to_upload = temp_hold_path 
folder_to_place_id = '1YXrIIXT3uSFd5-NSpht-1wB1-7hH6Y1w'
name_upload = 'Synthesis and UVVis'

CreateSamples.upload_to_team_drive_folder(folder_to_place_id, file_to_upload, name_upload)